In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import mibian
from datetime import datetime

In [7]:
url = 'https://www.meff.es/esp/Derivados-Financieros/Ficha/FIEM_MiniIbex_35'

In [4]:
def obtener_dataframes(url):
    url = 'https://www.meff.es/esp/Derivados-Financieros/Ficha/FIEM_MiniIbex_35'
    # Hacer la solicitud GET y obtener el contenido HTML
    response = requests.get(url)
    html = response.text
    
    # Crear un objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # Encuentra todas las filas de la tabla que contengan "OPE" o "OCE" en el atributo 'data-tipo'
    rows = soup.find_all('tr', {'data-tipo': lambda value: value and (value.startswith('OPE') or value.startswith('OCE'))})

    # Prepara listas para almacenar datos separados
    ope_data = []
    oce_data = []

    # Itera sobre las filas y extrae los datos
    for row in rows:
        cells = row.find_all('td')
        cell_data = [cell.get_text(strip=True) for cell in cells]
        # Extrae la fecha y el tipo
        data_tipo = row.get('data-tipo')
        data_type, date = data_tipo[:3], data_tipo[3:]

        # Clasifica los datos en OPE o OCE
        if data_type == 'OPE':
            ope_data.append([date] + cell_data)
        elif data_type == 'OCE':
            oce_data.append([date] + cell_data)

    # Crea DataFrames de pandas para cada conjunto de datos
    df_put = pd.DataFrame(ope_data)
    df_call = pd.DataFrame(oce_data)

    # Si la primera columna es la fecha, establecerla como índice
    df_put.set_index(0, inplace=True)
    df_call.set_index(0, inplace=True)

    # Cambia los valores vacíos por NaN y elimina filas con NaN
    df_put.replace({"": np.nan, "-": np.nan}, inplace=True)
    df_call.replace({"": np.nan, "-": np.nan}, inplace=True)
    df_put.dropna(subset=[df_put.columns[-1]], inplace=True)
    df_call.dropna(subset=[df_call.columns[-1]], inplace=True)

    # Selecciona las columnas de interés y renombra las columnas
    df_put = df_put.iloc[:, [0, -1]].copy()
    df_call = df_call.iloc[:, [0, -1]].copy()
    df_put.columns = ['Strike', 'Anterior']
    df_call.columns = ['Strike', 'Anterior']

    # Convierte las columnas a formato numérico
    df_put['Strike'] = df_put['Strike'].str.replace('.', '').str.replace(',', '.').astype(float)
    df_call['Strike'] = df_call['Strike'].str.replace('.', '').str.replace(',', '.').astype(float)
    df_put['Anterior'] = df_put['Anterior'].str.replace('.', '').str.replace(',', '.').astype(float)
    df_call['Anterior'] = df_call['Anterior'].str.replace('.', '').str.replace(',', '.').astype(float)

    # Convierte la fecha al formato adecuado y establece como índice
    df_put.index = pd.to_datetime(df_put.index, format='%Y%m%d')
    df_call.index = pd.to_datetime(df_call.index, format='%Y%m%d')
    
  
    # Extracción de datos de la tabla de futuros
    tabla_futuros = soup.find('table', { 'id':"Contenido_Contenido_tblFuturos"})
    futuros_list = []

    # Iterar sobre cada fila de la tabla excepto la cabecera
    for fila in tabla_futuros.find('tbody').find_all('tr', class_="text-right"):
        datos_fila = [celda.text.strip() for celda in fila.find_all('td')]
        futuro = {
                'Vencimiento' : datos_fila[0],
                'Anterior': datos_fila[13]
            }
        futuros_list.append(futuro)

    # Convertir la lista de diccionarios en un DataFrame de pandas
    df_futuros = pd.DataFrame(futuros_list)
    
    # Procesamiento adicional de futuros
    df_futuros['Anterior'] = df_futuros['Anterior'].str.replace('.', '').str.replace(',', '.').astype(float)
    df_futuros['Vencimiento'] = pd.to_datetime(df_futuros['Vencimiento'], format='%d %b. %Y')

    # Crear un DataFrame de opciones combinando calls y puts
    df_call['Tipo'] = 'Call'
    df_put['Tipo'] = 'Put'
    df_opciones = pd.concat([df_call, df_put])
    
    return df_opciones, df_futuros

In [8]:
df_opciones, df_futuros = obtener_dataframes(url)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_47840\2034154427.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_put.replace({"": np.nan, "-": np.nan}, inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_47840\2034154427.py:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_call.replace({"": np.nan, "-": np.nan}, inplace=True)


In [9]:
df_opciones

,Strike,Anterior,Tipo
0,,,
2024-05-03,11150.0,18.0,Call
2024-05-03,11175.0,15.0,Call
2024-05-03,11200.0,12.0,Call
2024-05-03,11275.0,6.0,Call
2024-05-03,11300.0,4.0,Call
...,...,...,...
2024-12-20,10500.0,352.0,Put
2024-12-20,10600.0,380.0,Put
2024-12-20,10700.0,411.0,Put


In [10]:
df_futuros

,Vencimiento,Anterior
0,2024-05-17,10931.1
1,2024-06-21,10949.0
2,2025-03-21,10951.0


In [11]:
def volatilidad_implicita_df(df, tabla_futuros):
    # Calcula la volatilidad implícita
    precio_subyacente = tabla_futuros['Anterior'].iloc[0]
    today_date = datetime.now()
    resultados = []

    for fecha, row in df.iterrows():
        strike = row['Strike']
        if row['Anterior'] == '-':
            continue
        precio_opcion = row['Anterior']
        tipo_de_opcion = row['Tipo']

        days_to_expiration = (pd.to_datetime(fecha) - today_date).days
        c = mibian.BS([precio_subyacente, strike, 0, days_to_expiration], callPrice=precio_opcion if tipo_de_opcion == 'Call' else None, putPrice=precio_opcion if tipo_de_opcion == 'Put' else None)
        resultados.append({
            'Fecha': fecha.strftime('%d-%m-%Y'),
            'Strike': strike,
            'Volatilidad': c.impliedVolatility,
            'Tipo': tipo_de_opcion
        })
    
    return pd.DataFrame(resultados)

In [12]:
resultados = volatilidad_implicita_df(df_opciones, df_futuros)

In [13]:
resultados

,Fecha,Strike,Volatilidad,Tipo
0,03-05-2024,11150.0,16.876221,Call
1,03-05-2024,11175.0,17.059326,Call
2,03-05-2024,11200.0,17.089844,Call
3,03-05-2024,11275.0,17.211914,Call
4,03-05-2024,11300.0,16.723633,Call
...,...,...,...,...
97,20-12-2024,10500.0,15.733719,Put
98,20-12-2024,10600.0,15.333176,Put
99,20-12-2024,10700.0,14.949799,Put
100,20-12-2024,11200.0,13.576508,Put
